In [1]:
from tkinter import *
from tkinter.ttk import Combobox
from PIL import Image, ImageTk
import requests
from bs4 import BeautifulSoup
import re

class CricketScore:
    def __init__(self, rootWindow):
        self.rootWindow = rootWindow
        self.rootWindow.title('LIVE CRICKET SCORE')
        self.rootWindow.geometry('900x550+100+50')  # Increased window size
        self.rootWindow.resizable(False, False)

        # Load Background Image
        try:
            self.bg_image = ImageTk.PhotoImage(Image.open("cric.jpg").resize((900, 550)))
            self.bg_label = Label(self.rootWindow, image=self.bg_image)
            self.bg_label.place(x=0, y=0, relwidth=1, relheight=1)
        except Exception as e:
            print("Error loading image:", e)

        # Title Bar
        self.title = Label(
            self.rootWindow,
            text="Live Cricket Scores",
            font=("Helvetica", 26, "bold"),
            bg="#0a9396",
            fg="white",
            padx=15,
            pady=10
        )
        self.title.place(relx=0.5, y=30, anchor=CENTER)

        # Match Selection Combobox
        self.var = StringVar()
        self.matches = self.match_details()
        self.data = list(self.matches.keys())
        self.cb = Combobox(
            self.rootWindow,
            values=self.data,
            textvariable=self.var,
            state="readonly",
            font=("Helvetica", 12)
        )
        self.cb.place(x=300, y=120, width=300, height=30)
        self.cb.set("Select a Match")

        # Check Score Button
        self.b1 = Button(
            self.rootWindow,
            text="Check Score",
            font=("Helvetica", 14, "bold"),
            bg="#ee9b00",
            fg="white",
            activebackground="#ca6702",
            activeforeground="white",
            relief=FLAT,
            command=self.show_match_details
        )
        self.b1.place(x=400, y=170, width=120, height=40)

    def show_match_details(self):
        selected_match = self.var.get()
        if selected_match not in self.matches:
            self.show_message("Please select a valid match!", "#d00000")
            return

        # Clear previous match details if any
        for widget in self.rootWindow.winfo_children():
            if isinstance(widget, Frame):
                widget.destroy()

        # Match Details Frame
        self.frame1 = Frame(self.rootWindow, bg="#f8f9fa", bd=3, relief=SOLID)
        self.frame1.place(x=150, y=250, width=700, height=200)  # Wider frame

        match_data = self.matches[selected_match]

        Label(
            self.frame1,
            text=f"{selected_match} - {match_data['match_header']}",
            font=("Helvetica", 16, "bold"),
            bg="#f8f9fa",
            fg="#005f73"
        ).place(x=10, y=10)

        Label(
            self.frame1,
            text=f"Score Details: {match_data['score_card']}",
            font=("Helvetica", 14),
            bg="#f8f9fa",
            fg="#005f73"
        ).place(x=10, y=70)

        Label(
            self.frame1,
            text=f"Summary: {match_data['summary']}",
            font=("Helvetica", 14),
            bg="#f8f9fa",
            fg="#005f73"
        ).place(x=10, y=150)

    def show_message(self, message, color):
        msg = Label(self.rootWindow, text=message, font=("Helvetica", 12), fg="white", bg=color, padx=10, pady=5)
        msg.place(relx=0.5, rely=0.95, anchor=S)
        self.rootWindow.after(3000, msg.destroy)

    def match_details(self):
        details = self.scrap()
        live_match = {}
        for detail in details:
            live_team_details = {}
            summary = self.match_header(detail)
            if summary:
                match_header = self.match_header(detail)
                teams = self.teams_name(detail)
                score_card = self.team_score(detail)
                live_team_details['summary'] = summary
                live_team_details['match_header'] = match_header
                live_team_details['score_card'] = score_card[0] + " :: " + score_card[1]
                live_match[f"{teams[0]} vs {teams[1]}"] = live_team_details
        return live_match

    def team_score(self, detail):
        t = []
        team1_details = self.safe_find(detail, "div", class_="cb-hmscg-bat-txt")
        team2_details = self.safe_find(detail, "div", class_="cb-hmscg-bwl-txt")
        t.append(team1_details)
        t.append(team2_details)
        return t

    def teams_name(self, detail):
        t = []
        team1_details = self.safe_find(detail, "div", class_="cb-hmscg-bat-txt")
        team2_details = self.safe_find(detail, "div", class_="cb-hmscg-bwl-txt")

        team1_index = re.search(r"\d", team1_details).start() if re.search(r"\d", team1_details) else len(team1_details)
        team2_index = re.search(r"\d", team2_details).start() if re.search(r"\d", team2_details) else len(team2_details)
        t.append(team1_details[:team1_index])
        t.append(team2_details[:team2_index])
        return t

    def match_header(self, detail):
        match_header = detail.find("div", class_="cb-mtch-crd-hdr")
        return match_header.text.strip() if match_header else "No Match Header"

    def safe_find(self, detail, tag, class_):
        element = detail.find(tag, class_=class_)
        return element.text.strip() if element else "Data not available"

    def scrap(self):
        try:
            URL = "https://www.cricbuzz.com/"
            page = requests.get(URL)
            soup = BeautifulSoup(page.content, "html.parser")
            results = soup.find(id="match_menu_container")
            return results.find_all("li", class_="cb-match-card")
        except Exception as e:
            print("Error scraping data:", e)
            return []


def main():
    rootWindow = Tk()
    CricketScore(rootWindow)
    rootWindow.mainloop()


if __name__ == '__main__':
    main()
